In [ ]:
from bs4 import BeautifulSoup
import requests
import pandas as pd
import csv

url = "https://books.toscrape.com/"

original = url
try:
    response = requests.get(url)
    response.raise_for_status()
except Exception as e:
    print(e)
    print("Error in getting the response")
    exit()

html_format = BeautifulSoup(response.content, "html.parser")

Book_dictionary = {
    "Book Name": [],
    "UPC": [],
    "Product Type": [],
    "Price(excl.tax)": [],
    "Price(Incl.Tax)": [],
    "Tax": [],
    "Availability": [],
    "Number of reviews": []
}
Book_Data_Frame = pd.DataFrame(Book_dictionary)
var=0

In [ ]:
#this will take 42.48 minutes to run
no_page=1
while True:
    try:
        link_list = []
        for link in html_format.find_all("h3"):
            link_list.append(link.a.get("href"))
        # print(link_list)
        for link2 in link_list:
            if(var==0):
                url2 = original + link2
                
            else:
                url2 = original +"catalogue/" + link2
                # print(url2)
            
            response2 = requests.get(url2)  
            response2.raise_for_status()
            html_format2 = BeautifulSoup(response2.content, "html.parser")
            Book_Name = html_format2.find("h1").text
            td_elements = html_format2.find_all("td")
            UPC = td_elements[0].text
            Product_Type = td_elements[1].text
            Price_excl_tax = td_elements[2].text
            Price_Incl_Tax = td_elements[3].text
            Tax = td_elements[4].text
            Availability = td_elements[5].text
            Number_of_reviews = td_elements[6].text
            Book_Data_Frame.loc[len(Book_Data_Frame)] = [Book_Name, UPC, Product_Type, Price_excl_tax, Price_Incl_Tax, Tax, Availability, Number_of_reviews]
        var+=1
        next_page = html_format.find("li", {"class": "next"})
        no_page+=1
        if next_page and next_page.a:
            next_url = next_page.a.get("href")
            #Their is url issue for next page in the website
            # print(next_url)
            if(next_url==f"page-{no_page}.html"):
                next_url=f"catalogue/page-{no_page}.html"
            url = original + next_url
            response = requests.get(url)
            response.raise_for_status()
            html_format = BeautifulSoup(response.content, "html.parser")
        else:
            break
    except Exception as e:
        print(e)
        break

print(Book_Data_Frame)


Book_Data_Frame.to_csv("Fake_BOOK_Data.csv")
